<a href="https://colab.research.google.com/github/Nolanole/DS-Unit-2-Sprint-3-Classification-Validation/blob/master/Tuesday_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ASSIGNMENT options

- **Replicate the lesson code.** [Do it "the hard way" or with the "Benjamin Franklin method."](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit)
- Apply the lesson to other datasets you've worked with before, and compare results.
- Iterate and improve your **Bank Marketing** model. Engineer new features.
- Get **weather** data for your own area and calculate both baselines.  _"One (persistence) predicts that the weather tomorrow is going to be whatever it was today. The other (climatology) predicts whatever the average historical weather has been on this day from prior years."_ What is the mean absolute error for each baseline? What if you average the two together? 
- [This example from scikit-learn documentation](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html) demonstrates its improved `OneHotEncoder` and new `ColumnTransformer` objects, which can replace functionality from [third-party libraries](https://github.com/scikit-learn-contrib) like category_encoders and sklearn-pandas. Adapt this example, which uses Titanic data, to work with Bank Marketing or another dataset.
- When would this notebook's pipelines fail? How could you fix them? Add more [preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html) and [imputation](https://scikit-learn.org/stable/modules/impute.html) to your [pipelines](https://scikit-learn.org/stable/modules/compose.html) with scikit-learn.

In [0]:
# Imports
%matplotlib inline
import warnings
import category_encoders as ce
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import graphviz
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [0]:
df = sns.load_dataset('titanic').drop(columns= ['alive', 'class', 'embark_town'])

In [0]:
#fillna for 2 missing rows of embarked, using most frequent embarkation:
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])

#fillna for age w/ ~mean age of 30:
df['age'] = df['age'].fillna(30)

In [18]:
#convert categorical cols to numeric cat codes:

cat_cols = ['pclass', 'sex', 'embarked', 'who', 'deck', 'adult_male', 'alone']

def convert_categoricals(df, cat_cols):
  copy = df.copy()
  for col in cat_cols:
    copy[col] = pd.Categorical(copy[col])
    copy[col] = copy[col].cat.codes
  return copy

df = convert_categoricals(df, cat_cols)
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,adult_male,deck,alone,num_companions
0,0,2,1,22.0,1,0,7.2500,2,1,1,-1,0,1
1,1,0,0,38.0,1,0,71.2833,0,2,0,2,0,1
2,1,2,0,26.0,0,0,7.9250,2,2,0,-1,1,0
3,1,0,0,35.0,1,0,53.1000,2,2,0,2,0,1
4,0,2,1,35.0,0,0,8.0500,2,1,1,-1,1,0


In [0]:
#try a lil extra feature_engineering:
df['num_companions'] = df['sibsp'] + df['parch'] 

#shift category by 1, so that lowest is 1 instead of 0:
df['embarked'] = df['embarked'] + 1
df['pclass'] = df['pclass'] + 1

#experimental features:
df['age_times_class'] = df['age']*(df['pclass'])
df['group_fare'] = df['fare']*(df['num_companions']+1)

In [20]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,adult_male,deck,alone,num_companions,age_times_class,group_fare
0,0,3,1,22.0,1,0,7.2500,3,1,1,-1,0,1,66.0,14.5000
1,1,1,0,38.0,1,0,71.2833,1,2,0,2,0,1,38.0,142.5666
2,1,3,0,26.0,0,0,7.9250,3,2,0,-1,1,0,78.0,7.9250
3,1,1,0,35.0,1,0,53.1000,3,2,0,2,0,1,35.0,106.2000
4,0,3,1,35.0,0,0,8.0500,3,1,1,-1,1,0,105.0,8.0500


In [0]:
def test_pipeline(pipeline, X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  
  pipeline.fit(X_train, y_train)
  y_pred = pipeline.predict(X_test)
  print('Accuracy score: ' + str(accuracy_score(y_pred, y_test)))
  y_pred_proba = pipeline.predict_proba(X_test)[:,1]
  print('Roc_auc_score: ' + str(roc_auc_score(y_test, y_pred_proba)))

In [0]:
tree_pipe = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    StandardScaler(),
    DecisionTreeClassifier()
)

log_reg_pipe = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    StandardScaler(),
    LogisticRegression(solver='lbfgs', max_iter=1000)
)

In [0]:
target = 'survived'
X = df.drop(columns=target)
y = df[target]

In [24]:
test_pipeline(tree_pipe, X, y)

Accuracy score: 0.7541899441340782
Roc_auc_score: 0.7658944658944659


In [25]:
test_pipeline(log_reg_pipe, X, y)

Accuracy score: 0.8100558659217877
Roc_auc_score: 0.8801801801801802


In [0]:
#Use cross-validation w/ training data instead:

def test_pipeline_cross_val(pipeline, X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  scores = cross_val_score(pipeline, X_train, y_train, scoring='roc_auc', cv=10, n_jobs=-1, verbose=10)
  print('Cross-Validation ROC AUC scores:', scores)
  print('Average:', scores.mean())

In [27]:
test_pipeline_cross_val(tree_pipe, X, y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Cross-Validation ROC AUC scores: [0.8037037  0.72962963 0.71152263 0.79876543 0.79040404 0.68013468
 0.78493266 0.78493266 0.74300699 0.8784965 ]
Average: 0.7705528936084491


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.4s finished


In [28]:
test_pipeline_cross_val(log_reg_pipe, X, y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0545s.) Setting batch_size=6.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished


Cross-Validation ROC AUC scores: [0.92263374 0.83786008 0.70452675 0.95144033 0.88636364 0.81060606
 0.76346801 0.89646465 0.82692308 0.96416084]
Average: 0.8564447178336068
